In [1]:
import numpy as np
from utils_predictions_manipulation import*
from utils_nans_manipulation import*
from cross_validation import*
from utils_data_loading import*
from utils_features_manipulation import*
import random
import sys, os
from time import time

In [2]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

## Load data

In [3]:
traindata,_ = load_data('Data/train.csv')
X_total, Y_total = structure_data(traindata)

# Replacing undefined data with NaNs
X_nans = replace_bad_data_with_nans(X_total, -999)

X_nans, col = replace_nans_with_median(X_nans, threshold=0.5)

X_nans = standardize_data(X_nans)
#X_nans = X_nans + np.random.rand(X_nans.shape[0], X_nans.shape[1])/10000

In [49]:
#Baseline
start = time()
with HiddenPrints():
    dtmp_tr,dtmp_te = cross_validation(Y_total, X_nans, 
                                               k_fold=4, seed=1, 
                                               function_name='logistic_regression',
                                               max_iters=60, gamma=5e-7)

end = time()

print(end-start)
dtmp_tr

1.8342430591583252


0.7029653333333333

In [5]:
# From LS optimization
ind_v = (1, 3, 5, 7, 13, 16, 18, 20, 23) 
deg_v = [(0.25, 0.3333333333333333, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.1, 0.3333333333333333, 20), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.05, 10), [0.25, 3, 4, 0.3333333333333333], (3, 10, 20), (0.1, 0.3333333333333333, 4), (0.03333333333333333, 0.2, 1), (0.1, 2, 5), (0.05, 0.1, 1), (0.2, 0.3333333333333333, 0.5, 2, 4, 10), (0.1, 0.25, 0.3333333333333333, 1, 5, 30), (1, 3, 30), (0.1, 5, 10), (0.25, 3, 30), (0.1, 10, 30), (0.2, 1, 3), (0.05, 0.5, 30), (1, 3, 4), (0.3333333333333333, 3, 5), (2, 10), (0.25, 0.3333333333333333, 1), (0.05, 0.2, 30)]
        
X_poly = build_poly_index(X_nans, ind_v, deg_v)

with HiddenPrints():
    dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, 
                                               k_fold=4, seed=1, 
                                               function_name='logistic_regression',
                                               max_iters=60, gamma=5e-7)
dtmp_tr

/Users/ENVasquez/Desktop/ML_Project_1/utils_features_manipulation.py:48: RuntimeWarning: invalid value encountered in power
  coltmp=tx[:,i]**d


0.657332

## Setup genetic pool

In [50]:
degree_test = [1/5, 1/4, 1/3, 1/2, 1, 2, 3, 4, 5]
deg_all = all_combinations_list(degree_test)


In [51]:
index_test = list(range(1,X_nans.shape[1]))
#index_remove = [9,11,12,14,15,17,21,22]
#index_test = list(set(index_test)-set(index_remove))
ind_all = all_combinations_list(index_test)

In [52]:
tot = (1/6)*60*60 / 2.5 # 6h runtime
size = 50
days = int(tot/size)
print(days)

4


In [53]:
#ind_pool = [(1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 2, 16, 18, 19, 23), (2, 4, 5, 7, 13, 16, 18, 20, 23), (4, 6, 7, 19, 23), (2, 4, 7, 10, 13, 20, 23), (1, 2, 3, 6, 7, 8, 10, 13, 16, 18, 19, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 4, 5, 6, 10, 18, 19, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23), (1, 3, 5, 7, 13, 16, 18, 20, 23)]
#deg_pool = [[(0.25, 2, 20), (0.25, 2, 3), (0.25, 2, 20), (0.25, 0.3333333333333333, 2), (0.2, 0.25, 30), (0.05, 0.5, 20), (0.25, 2, 3), (0.25, 0.3333333333333333, 2), (0.03333333333333333, 0.1, 5), (0.25, 2, 20), (0.25, 0.3333333333333333, 0.5), (0.2, 1, 4), (0.2, 1, 4), (0.25, 0.3333333333333333, 2), (0.25, 2, 3), (0.5, 4, 10), (0.1, 3, 5), (1, 5), (0.05, 1, 10), (0.5, 5), (0.25, 2, 3), (0.03333333333333333, 0.1, 5), (0.25, 2, 3), (0.03333333333333333, 0.1, 5), (0.25, 0.3333333333333333, 2)], [(0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.3333333333333333, 0.5, 2), (0.1, 5, 10), (0.03333333333333333, 0.5, 20), (0.5, 4, 10), (3, 5, 10), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2), (0.03333333333333333, 0.1, 0.2, 0.3333333333333333, 0.5, 2)], [(0.3333333333333333, 5), (0.1, 0.3333333333333333, 20), (0.1, 0.3333333333333333, 20), (0.05, 0.3333333333333333, 4), (0.1, 0.3333333333333333, 20), (0.1, 5, 20), (0.1, 0.3333333333333333, 20), (4, 5), (0.1, 0.3333333333333333, 20), (0.1, 0.3333333333333333, 20), (0.05, 3, 5), (0.5, 5, 20), (0.5, 2, 5), (0.1, 0.3333333333333333, 20), (0.5, 2, 20), (2, 10), (0.03333333333333333, 2, 4), (0.5, 3, 5), (5, 10, 20), (0.1, 0.2, 1), (0.5, 2, 20), (0.1, 5, 20), (0.1, 0.3333333333333333, 20), (0.5, 5, 20), (0.1, 0.3333333333333333, 20)], [(0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.5, 1, 3), (0.25, 0.5, 3), (2, 4, 30), (0.5, 20, 30), (0.25, 0.5), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30), (0.03333333333333333, 0.05, 0.2, 0.3333333333333333, 30)], [(0.05, 10), (0.25, 3, 30), (0.05, 0.3333333333333333, 10), (0.05, 0.3333333333333333, 10), (0.1, 3, 30), (0.25, 2, 3, 5, 20, 30), (0.25, 2, 3, 5, 20, 30), (0.3333333333333333, 0.5), (0.05, 10), (0.05, 10), (0.25, 3, 30), (0.05, 0.1, 0.5), (0.05, 10), (0.3333333333333333, 0.5), (0.05, 0.3333333333333333, 10), (0.05, 0.1, 30), (5, 10, 20), (1, 2, 30), (0.2, 0.3333333333333333, 4), (3, 4, 10), (0.05, 0.3333333333333333, 10), (0.25, 2, 3, 5, 20, 30), (0.05, 10), (0.25, 2, 3, 5, 20, 30), (0.05, 10)], [(0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.25, 0.3333333333333333, 20), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.05, 20), (0.03333333333333333, 0.5, 4), (0.25, 1), (0.25, 5, 10), (0.05, 0.1, 10), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5), (0.2, 0.25, 0.3333333333333333, 3, 4, 5)], [(0.05, 0.2, 0.3333333333333333), (0.05, 0.2, 0.3333333333333333), (0.1, 1), (2, 5), (0.1, 4), (3, 10, 20), (2, 30), (0.2, 0.3333333333333333, 30), (0.3333333333333333, 2, 30), (0.5, 1, 20), (3, 10, 20), (2, 5), (0.25, 20, 30), (3, 10, 20), (0.05, 0.2, 0.3333333333333333), (0.1, 0.5, 10), (0.05, 0.5, 30), (0.05,), (0.25, 10), (2, 4, 20), (0.05, 0.2, 0.3333333333333333), (2, 5), (0.25, 3, 5), (0.05, 0.2, 0.3333333333333333), (3, 10, 20)], [(0.03333333333333333, 1, 30), (0.25, 5), (0.5, 1, 5), (0.03333333333333333, 0.5, 10), (0.5, 1, 5), (0.1, 0.3333333333333333, 4), (0.03333333333333333, 2, 3), (0.03333333333333333, 0.2, 10), (0.05, 3, 20), (0.03333333333333333, 0.2, 10), (0.03333333333333333, 0.2, 10), (0.03333333333333333, 0.2, 10), (0.05, 3, 20), (0.03333333333333333, 0.2, 10), (0.3333333333333333, 1, 2), (0.25, 1, 5), (0.05, 5, 20), (0.05, 2, 5), (0.03333333333333333, 0.05, 30), (4, 5, 30), (0.3333333333333333, 1, 2), (0.3333333333333333, 1, 2), (0.03333333333333333, 0.2, 10), (0.03333333333333333, 0.5, 10), (0.1, 0.3333333333333333, 4)], [(0.2, 5, 30), (0.03333333333333333, 0.2, 1), (0.2, 5, 30), (1, 3, 4), (0.03333333333333333, 0.2, 1), (0.03333333333333333, 0.2, 1), (0.2, 5, 30), (1, 10, 20), (10,), (0.2, 5, 30), (0.1, 0.3333333333333333, 3), (0.03333333333333333, 0.2, 1), (0.2, 5, 30), (0.2, 5, 30), (0.03333333333333333, 0.2, 1), (2, 20, 30), (1, 2, 30), (0.05, 1, 30), (0.03333333333333333, 0.5, 5), (3, 10, 30), (0.03333333333333333, 0.2, 1), (0.2, 5, 30), (0.03333333333333333, 0.2, 1), (0.03333333333333333, 0.2, 1), (0.03333333333333333, 0.2, 1)], [(0.25, 0.3333333333333333), (0.2, 2, 20), (0.1, 0.3333333333333333, 10), (0.1, 2, 5), (3, 10, 30), (0.2, 2, 20), (4, 5, 20), (0.2, 2, 20), (0.05, 0.1, 4), (0.25, 1, 5), (0.05, 0.1, 4), (0.05, 0.1, 4), (3, 10, 30), (0.03333333333333333, 30), (0.2, 2, 20), (0.3333333333333333, 0.5), (0.2, 0.3333333333333333), (0.05, 3), (0.05, 0.25, 3), (0.05, 1, 2), (0.2, 2, 20), (0.1, 2, 5), (0.2, 2, 20), (0.03333333333333333, 30), (0.1, 2, 5)], [(4, 5, 20), (0.25, 3, 20), (0.1, 0.3333333333333333, 20), (0.05, 0.5, 1), (1, 2, 5), (0.25, 3, 20), (0.05, 3, 4), (1, 2, 5), (0.03333333333333333, 0.1, 30), (0.1, 0.3333333333333333, 20), (0.05, 0.1, 1), (0.05, 0.1, 1), (0.05, 0.25, 3), (0.05, 0.5), (0.05, 0.25, 3), (0.2, 0.3333333333333333, 2), (3, 5, 10), (0.1,), (0.03333333333333333, 0.5, 5), (0.2, 10), (0.05, 0.25, 3), (0.05, 0.25, 3), (0.05, 0.1, 1), (0.1, 0.3333333333333333, 20), (0.05, 0.1, 1)], [(0.2, 0.3333333333333333, 0.5, 2, 4, 10), (0.25, 30), (0.2, 0.3333333333333333, 0.5, 2, 4, 10), (0.1, 0.3333333333333333, 4), (0.5, 30), (2, 20, 30), (0.2, 0.3333333333333333, 0.5, 2, 4, 10), (2, 20, 30), (0.25, 1, 20), (0.05, 0.3333333333333333, 20), (0.25, 1, 20), (0.25, 1, 20), (0.3333333333333333, 4, 10), (0.5, 30), (0.1, 0.3333333333333333, 4), (0.03333333333333333, 0.3333333333333333, 5), (3, 4, 20), (0.05, 0.5, 20), (0.03333333333333333, 10), (0.2, 0.25, 0.5), (0.1, 0.3333333333333333, 4), (0.2, 0.3333333333333333, 0.5, 2, 4, 10), (0.25, 1, 20), (0.25, 1, 20), (0.2, 0.3333333333333333, 0.5, 2, 4, 10)], [(0.05, 0.1, 0.2), (0.2, 1, 10), (0.5, 3, 10), (0.05, 0.5, 2), (0.05, 0.5, 2), (0.03333333333333333, 0.3333333333333333, 1), (0.2, 1, 10), (0.1, 0.25, 0.3333333333333333, 1, 5, 30), (0.05, 0.5, 2), (0.5, 3, 10), (0.5, 1, 3), (0.05, 0.5, 2), (0.1, 0.25, 0.3333333333333333, 1, 5, 30), (0.05, 0.1, 0.2), (0.1, 0.25, 0.3333333333333333, 1, 5, 30), (0.1, 0.25, 0.5), (0.5, 3, 4), (0.25, 4, 20), (0.1, 0.2, 2), (0.2, 0.5, 3), (0.1, 0.25, 0.3333333333333333, 1, 5, 30), (0.1, 0.25, 0.3333333333333333, 1, 5, 30), (0.03333333333333333, 0.2, 0.25, 1, 10), (0.05, 0.5, 2), (0.1, 0.25, 0.3333333333333333, 1, 5, 30)], [(1, 3, 30), (1, 3, 30), (0.3333333333333333, 4, 10), (1, 3, 30), (0.2, 1, 4), (0.5, 1, 30), (1, 3, 30), (1, 3, 30), (0.3333333333333333, 4, 10), (0.2, 1, 4), (0.3333333333333333, 4, 10), (1, 3, 30), (1, 3, 30), (1, 3, 30), (0.25, 2, 20), (0.03333333333333333, 3, 10), (0.2, 1), (0.1, 0.3333333333333333, 1), (1, 5, 20), (0.03333333333333333, 20, 30), (0.25, 2, 20), (0.1, 0.2, 1, 3, 4, 10), (0.3333333333333333, 4, 10), (1, 3, 30), (1, 3, 30)], [(2, 4, 20), (0.1, 0.5, 1), (0.03333333333333333, 0.5, 3, 30), (2, 4, 20), (0.05, 3, 4), (0.2, 0.25, 0.5), (0.05, 3, 4), (0.5, 5, 10), (0.2, 0.25, 0.5), (0.1, 5, 10), (0.2, 0.25, 4), (0.05, 3, 4), (0.1, 5, 10), (0.03333333333333333, 0.5, 3, 30), (0.1, 5, 10), (0.25, 4, 5), (0.1, 20), (1,), (0.2, 4, 30), (0.03333333333333333, 0.2, 3), (0.1, 5, 10), (0.03333333333333333, 0.5, 3, 30), (0.03333333333333333, 0.5, 3, 30), (0.2, 0.25, 0.5), (0.1, 5, 10)], [(10, 20, 30), (0.25, 5), (0.2, 0.3333333333333333), (0.25, 5), (0.03333333333333333, 0.3333333333333333, 30), (0.03333333333333333, 0.25, 1), (0.25, 5), (0.25, 3, 30), (0.03333333333333333, 0.2, 20), (0.25, 5), (0.03333333333333333, 0.1, 4), (0.25, 5), (0.25, 3, 30), (0.2, 0.3333333333333333), (0.2, 0.3333333333333333), (0.1, 0.3333333333333333, 20), (4, 10), (0.1, 0.3333333333333333, 10), (0.05, 0.2, 2), (0.05, 0.25, 5), (0.2, 0.3333333333333333), (0.25, 3, 30), (0.25, 3, 30), (0.25, 5), (0.25, 3, 30)], [(0.1, 1, 3, 4, 10, 30), (0.05, 4), (0.05, 4), (0.05, 4), (0.1, 10, 30), (0.05, 4), (0.05, 4), (0.05, 4), (0.1, 10, 30), (0.05, 4), (0.05, 4), (0.05, 4), (0.05, 4), (0.05, 4), (0.1, 10, 30), (0.25, 1, 2), (0.25, 1, 3), (0.2, 10, 20), (0.03333333333333333, 0.2, 20), (3, 5, 20), (0.1, 10, 30), (0.1, 1, 3, 4, 10, 30), (0.1, 10, 30), (0.1, 10, 30), (0.1, 10, 30)], [(0.05, 0.25, 0.3333333333333333), (0.05, 0.1, 3), (0.05, 0.25, 0.3333333333333333), (0.2, 0.3333333333333333, 2), (0.03333333333333333, 0.05, 2), (0.5, 2, 4), (0.03333333333333333,), (2,), (0.2, 1, 3), (0.03333333333333333,), (0.03333333333333333, 2, 4), (0.03333333333333333, 0.05, 2), (0.2, 0.3333333333333333, 2), (0.25, 0.5, 20), (0.03333333333333333, 2, 4), (0.1, 0.2, 3), (0.5, 3, 20), (0.1, 0.3333333333333333, 2), (2,), (0.03333333333333333, 5, 30), (0.03333333333333333, 2, 4), (1, 2, 20), (0.03333333333333333, 2, 4), (0.05, 0.1, 3), (0.2, 1, 3)], [(0.05, 0.5, 30), (0.03333333333333333, 0.25, 0.3333333333333333), (0.03333333333333333, 0.25, 0.3333333333333333), (0.03333333333333333, 0.5, 3), (0.5, 2, 5), (0.5, 2, 5), (0.2, 0.5, 10), (0.05, 0.5, 30), (0.05, 2, 5), (0.05, 0.5, 30), (0.05, 0.5, 30), (0.05, 0.5, 30), (0.05, 0.5, 30), (0.05, 0.5, 30), (0.05, 0.5, 30), (0.05, 0.1, 0.3333333333333333), (0.1, 3, 5), (0.05, 4, 30), (1, 10), (0.05, 3, 20), (0.05, 0.5, 30), (0.5, 1, 4, 10, 30), (0.05, 0.5, 30), (0.05, 0.5, 30), (0.05, 0.5, 30)], [(0.03333333333333333, 0.2, 0.5, 1, 3, 5), (0.03333333333333333, 0.2, 0.5, 1, 3, 5), (0.25, 30), (0.2, 2, 30), (0.25, 30), (0.03333333333333333, 0.2, 0.5, 1, 3, 5), (0.5,), (0.03333333333333333, 0.2, 0.5, 1, 3, 5), (0.25, 30), (0.2, 2, 30), (0.03333333333333333, 0.05, 30), (0.25, 30), (1, 3, 4), (0.05, 30), (1, 3, 4), (0.3333333333333333, 3), (0.05, 0.1, 5), (0.2, 10, 20), (0.05, 0.1, 5), (1, 10), (1, 3, 4), (0.03333333333333333, 0.2, 0.5, 1, 3, 5), (0.5, 4, 5), (0.03333333333333333, 0.2, 0.5, 1, 3, 5), (1, 3, 4)], [(0.3333333333333333, 2), (0.1, 5, 30), (0.1, 5, 30), (0.3333333333333333, 3, 5), (0.05, 0.25, 0.5), (0.05, 0.25, 0.5), (0.3333333333333333, 2), (0.03333333333333333, 0.05, 1), (0.03333333333333333, 0.1, 0.2, 0.25, 2, 4), (0.03333333333333333, 0.1, 0.2, 0.25, 2, 4), (0.03333333333333333, 0.1, 0.2, 0.25, 2, 4), (0.03333333333333333, 0.05, 1), (0.3333333333333333, 3, 5), (0.03333333333333333, 0.05, 0.5), (0.3333333333333333, 2), (0.03333333333333333, 2, 20), (0.03333333333333333, 0.3333333333333333, 5), (0.25, 0.5, 3), (0.05, 0.5, 4), (0.03333333333333333, 1, 2), (0.3333333333333333, 2), (0.03333333333333333, 0.1, 0.2, 0.25, 2, 4), (0.3333333333333333, 2), (0.03333333333333333, 0.05, 1), (0.3333333333333333, 3, 5)], [(0.1, 0.2, 20), (0.1, 0.2, 20), (0.25, 0.5), (0.1, 0.2, 20), (5, 10), (0.05, 2, 4), (0.25, 0.5), (0.05, 4, 30), (0.1, 0.2, 20), (0.05, 4, 10), (0.05, 2, 4), (0.05, 4, 10), (2, 10), (0.05, 4, 10), (0.3333333333333333, 0.5, 5), (0.2, 1), (0.3333333333333333, 1, 30), (1, 10, 30), (0.03333333333333333,), (0.2, 0.25, 5), (0.3333333333333333, 0.5, 5), (0.3333333333333333, 30), (0.05, 2, 4), (0.05, 2, 4), (2, 10)], [(0.03333333333333333, 0.1, 1), (0.05, 0.1, 0.25, 0.3333333333333333, 1, 2), (0.05, 0.1, 0.25, 0.3333333333333333, 1, 2), (0.05, 1, 3), (0.1, 4, 10), (0.2, 0.25, 5), (0.2, 0.25, 5), (0.25, 0.3333333333333333, 1), (0.1, 4, 10), (0.05, 0.1, 0.25, 0.3333333333333333, 1, 2), (0.2, 0.25, 5), (0.2, 0.25, 5), (0.05, 0.1, 0.25, 0.3333333333333333, 1, 2), (0.05, 0.1, 0.25, 0.3333333333333333, 1, 2), (0.25, 0.3333333333333333, 1), (0.3333333333333333, 0.5, 30), (0.2, 3, 20), (0.25, 3, 5), (0.2, 0.5, 3), (0.25, 2, 3), (0.25, 0.3333333333333333, 1), (0.05, 0.1, 0.25, 0.3333333333333333, 1, 2), (0.05, 0.1, 0.25, 0.3333333333333333, 1, 2), (0.1, 4, 10), (0.25, 0.3333333333333333, 1)], [(0.1, 0.2, 30), (0.2, 4), (0.05, 0.1, 4), (0.05, 0.2, 30), (0.05, 0.2, 30), (0.05, 0.2, 30), (0.05, 0.2, 30), (0.2, 4), (0.05, 0.2, 30), (0.05, 0.2, 30), (0.05, 0.1, 4), (0.05, 0.1, 4), (0.05, 0.2, 30), (0.05, 0.2, 30), (0.05, 0.2, 30), (0.05, 0.25, 4), (0.25, 0.5), (0.05, 2, 10), (0.25, 2, 4), (0.03333333333333333, 0.3333333333333333, 0.5), (0.05, 0.2, 30), (0.2, 4), (0.05, 0.1, 4), (0.05, 0.2, 30), (0.05, 0.2, 30)]]


In [54]:
champ_surv = 5 #Can be greater than size
pop_surv = 15
mutations = 5

complexity_deg = 120
complexity_ind = 1000

deg_all2 = [deg_all[0:complexity_deg] for i in range(X_nans.shape[1])] #Store as chromosomes

#Initialize

deg_pool = deg_all2.copy()
ind_pool = ind_all[0:complexity_ind]

champions = []

start = time()
for day in range(days):
    print("\n Day:", day)
    
    #Generate population
    population = []
    for i in range(size):
        expression = random.choice(ind_pool)
        genes = [random.choice(chromosome) for chromosome in deg_pool]
        population.append([expression, genes])
    
    #Test reproductive success
    success = []
    for individual in population:
        X_poly = build_poly_index(X_nans, individual[0], individual[1])
        
        with HiddenPrints():
            dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, 
                                                       k_fold=4, seed=1, 
                                                       function_name='logistic_regression',
                                                       max_iters=60, gamma=5e-7)
        success.append(dtmp_tr)
        print(len(individual[0]), len(individual[1]), dtmp_tr)
    print("Average", np.mean(success))
    
    #Select best performing member for posteriory
    champions.append([max(success), population[success.index(max(success))]])
    
    #Select top 10% individuals
    survive = np.argsort(success)[-pop_surv:]
    survivors = [population[live] for live in survive]
    
    #Extract their features
    ind_pool = [individual[0] for individual in survivors]
    deg_pool = [[individual[1][i] for individual in survivors] for i in range(len(deg_pool))].copy()
    
    #Add fresh material to the gene pool
    for i in range(mutations):
        for chromosome in deg_pool:
            chromosome.append(random.choice(deg_all2[0]))
        ind_pool.append(random.choice(ind_all[0:complexity_ind]))
    
    #Select best champions:
    if len(champions)>champ_surv:
        best_champs = np.argsort([ind[0] for ind in champions])[-champ_surv:]
        champions = [champions[i] for i in best_champs]
    print(len(champions))
    
    #Add champion values to pool
    ind_pool.extend([ind[1][0] for ind in champions])
    for i, chromosome in enumerate(deg_pool):
        individual_genes = [ind[1][1] for ind in champions]
        chromosome.extend([genes[i] for genes in individual_genes])
    #print(len(deg_pool[0]), deg_pool[0])
    
    end = time()
    print(end-start)


 Day: 0
3 24 0.657332
3 24 0.657332
3 24 0.6874399999999999
3 24 0.657332
3 24 0.6956533333333332
3 24 0.7030933333333333
3 24 0.6974226666666667
3 24 0.657332
1 24 0.657332
2 24 0.6986866666666667
3 24 0.657332
2 24 0.6630466666666667
3 24 0.6301586666666666
2 24 0.657332
3 24 0.7119946666666667
2 24 0.657332
3 24 0.657332
3 24 0.7036133333333333
3 24 0.657332
2 24 0.6686000000000001
2 24 0.6820986666666666
3 24 0.657332
3 24 0.657332
3 24 0.657332
3 24 0.604032
3 24 0.6692946666666666
3 24 0.657332
3 24 0.657332
2 24 0.7142613333333333
3 24 0.7041573333333333
3 24 0.628588
2 24 0.654112
3 24 0.6495533333333333
3 24 0.696324
3 24 0.5197253333333334
3 24 0.7033093333333333
3 24 0.657332
3 24 0.6134786666666667
2 24 0.6732933333333334
3 24 0.5649586666666666
3 24 0.657332
3 24 0.712296
3 24 0.647124
3 24 0.657332
2 24 0.657332
3 24 0.6540346666666667
3 24 0.657332
3 24 0.657332
3 24 0.6123453333333333
2 24 0.657332
Average 0.6604799999999998
1
114.8354709148407

 Day: 1
3 24 0.657332
3

In [ ]:
for ind in champions:
    print(ind[0])

In [ ]:
ch_ind = champions[4][1][0]
ch_deg = champions[4][1][1]
accu = champions[4][0]

In [ ]:
for ind in ch_ind:
    print(ind, ch_deg[ind])

In [ ]:
ch_degm = ch_deg.copy()
ch_degm[1]

In [ ]:
ch_degm[1][5]

In [ ]:
#Dropping 1
for ind in ch_ind:
    
    for j in range(len(ch_degm[ind])):
        ch_degm = ch_deg.copy()
        ch_degm[ind] = list(set(ch_degm[ind]) - set([ch_degm[ind][j]]))
        
        X_poly = build_poly_index(X_nans, ch_ind, ch_degm)
        dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, k_fold=4, seed=1, function_name='least_squares')
        print(ind, j, dtmp_tr, dtmp_tr>=accu)
    

In [ ]:
#Dropping 2
for ind in ch_ind:
    
    for j in range(len(ch_degm[ind])):
        ch_degm = ch_deg.copy()
        ch_degm[ind] = list(set(ch_degm[ind]) - set([ch_degm[ind][j]]))
        ch_ori = ch_degm[ind].copy()
        print(ind, j)
        for k in range(len(ch_degm[ind])):
            ch_degm[ind] = ch_ori.copy()
            ch_degm[ind] = list(set(ch_degm[ind]) - set([ch_degm[ind][k]]))
            X_poly = build_poly_index(X_nans, ch_ind, ch_degm)
            dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, k_fold=4, seed=1, function_name='least_squares')
            print("\t", ind, k, dtmp_tr, dtmp_tr>=accu)

In [ ]:
ch_degm = ch_deg.copy()
ind = 5
ch_degm[ind] = list(set(ch_degm[ind]) - set([ch_degm[ind][0]]))
ch_degm[ind] = list(set(ch_degm[ind]) - set([ch_degm[ind][4]]))

X_poly = build_poly_index(X_nans, ch_ind, ch_degm)
dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, k_fold=4, seed=1, function_name='least_squares')
print("\t", ind, k, dtmp_tr, dtmp_tr>=accu)


In [ ]:
ch_indm = ch_ind
for ind in ch_ind:
    ch_indm = ch_ind
    ch_indm = list(set(ch_indm) - {ind})
    X_poly = build_poly_index(X_nans, ch_indm, ch_degm)
    dtmp_tr,dtmp_te = cross_validation(Y_total, X_poly, k_fold=4, seed=1, function_name='least_squares')
    print("\t", ind, dtmp_tr, dtmp_tr>=accu)

In [ ]:
print(ch_ind, ch_degm)

In [ ]:
print(ind_pool)

In [ ]:
print(deg_pool)

# Stored selections

In [ ]:
print(third_selection)

In [ ]:
print(second_selection)

In [ ]:
print(first_selection)

# Debugging

In [ ]:
def build_poly_index(tx, index_list, degree):
    """polynomial basis functions for input data tx, for all features expanded to d degree.Extended matrix is inputed np.array. Index list includes all variables that will be expanded, mostly useful to avoid expanding categorical variables"""
    # ***************************************************
    # augmented x matrix with column of 1 input
    #list of indices for features modified
    #degrees of polynomial to apply to all features irrespectively
    # ***************************************************
    added_cols = sum([(len(degree[i])-1) for i in index_list])
    xmat=np.empty((tx.shape[0], tx.shape[1] + sum([(len(degree[i])-1) for i in index_list])))
    ind = 0
    for i in range(tx.shape[1]):
        if i in index_list:
            #print("BP:", i, degree[i])
            for d in degree[i]:
                coltmp=tx[:,i]**d
                xmat[:,ind] = coltmp
                ind+=1
        else:
            coltmp=tx[:,i]
            xmat[:,ind] = coltmp
            ind+=1
    return xmat

In [ ]:
index_list = list(range(X_nans.shape[1]))

degree_test = [1/30, 1/20, 1/10, 1/5, 1/4, 1/3, 1/2, 1, 2, 3, 4, 5, 10, 20, 30]
degree = [degree_test for i in index_list]

index_list = [3,23]

out = build_poly_index(X_nans, index_list, degree)
out.shape

In [ ]:
X_nans[:,0]

In [ ]:
out[:,0]